# Stitch sequence from pieces

You are given pieces (subarrays) from an array. Reconstruct the array. For example: (1,2,4) , (1,2,5), (4,5) --> (1,2,4,5)

In [3]:
class Solution():
    def __init__(self):
        self.g = {}     # Graph. self.g[i] contains a list of all indices to which i is connected
        self.trace = {} # Trace[i] will contain the highest distance from the root. Is it smelly to have it global?
    
    def add_piece(self, x):
        """Adds a fragment"""
        for i in range(len(x)):
            if x[i] not in self.g:
                self.g[x[i]] = []
            if i<len(x)-1:
                if x[i+1] not in self.g[x[i]]:
                    self.g[x[i]].append(x[i+1]) # Just add the vertices
                    
    def parse_sequence(self,x):
        """Parse a list of graments"""
        for part in x:
            self.add_piece(part)
            
    def source(self):
        """Find the source for a graph (very slow)"""
        p = list(s.g.keys())[0]                # Grab one node at random
        flag = True                            # Try while it works
        while flag:
            flag = False
            for (k,v) in self.g.items():       # Cycle through all node
                if p in v:                     # If any node links to this one, move upstream
                    p = k
                    flag = True                # Repeat until it works.
        return p                               # Reversing the graph and doing DFS would have been much faster.
    
    def prune(self,p=None,lvl=0):
        """Traverse and prune DFS."""
        if p is None:
            p = self.source()                  # If root wasn't given, find it
        if p not in self.trace:                # Set level of root at 0, level of all other first visited nodes at lvl
            self.trace[p] = lvl
        if lvl>self.trace[p]:                  # If the node was visited before, update level.
            self.trace[p] = lvl                # The later we visit - the better! We are after longest paths.
        for k in self.g[p]:
            self.prune(k,lvl+1)                # Recurse (DFS).
            
    def traverse(self,p=None):
        """Use trace to traverse properly"""
        if p is None:
            p = self.source()                           # Starting from the root..
        if len(self.trace)==0:
            self.prune(p)                               # If levels weren't calculated yet, calculate them
        levels = [self.trace[i] for i in self.g[p]]     # Find distances of all nodes reachable from this node
        if levels==[]:                                  # If dead-end, return the note itself
            return [p]
        step = self.g[p][levels.index(min(levels))]     # Otherwise, find next node (one with smallest distance)
        return [p] + self.traverse(step)                # Return current node + next node recursively
        
                
# test
x = [[1,2,4],[1,2,5],[4,5],[0,2],[0,1,4]]
s = Solution()
s.parse_sequence(x)
# print(s.g)
print(s.traverse())


[0, 1, 2, 4, 5]


I'm not particularly happy about this solution. Finding the root can be done faster (reverse the graph + DFG?), having a separate global dictionary to mark distances to nodes (`trace`) feels wrong (smally code? It's hard to keep it all in mind, that's for sure, which is not a good sign). Also, there's perfectly 0 protection against cycles. If there's a cycle, the `trace` calculator will just loop forever.

In [5]:
import numpy as np
import random as rnd

def generate():
    n_length = 30
    source = list(range(n_length))
    n_pieces = n_length
    x = []
    for i_piece in range(n_pieces):
        i_length = np.random.randint(2,n_length // 2)
        y = np.sort(rnd.sample(source,i_length)).tolist()
        x.append(y)
    return x

n_trials = 1
for i_trial in range(n_trials):
    x = generate()
    print(x)
    s = Solution()
    s.parse_sequence(x)
    print(s.traverse())

[[8, 16, 29], [8, 9, 12, 20, 24], [7, 13, 15, 27], [18, 24, 25], [1, 2, 3, 5, 10, 13, 15, 16, 18, 19, 27, 28, 29], [12, 26, 28], [0, 2, 4, 5, 6, 8, 10, 12, 21, 24, 26, 28, 29], [0, 21], [4, 14, 15, 20, 23, 27], [1, 5, 15, 19, 27, 29], [0, 4, 5, 6, 10, 12, 13], [0, 2, 4, 8, 9, 11, 13, 17, 18, 20, 21, 22, 26, 29], [8, 9], [0, 3, 7, 12, 16], [5, 7, 14, 19, 20, 21, 22, 24, 28, 29], [3, 5, 10, 15, 22], [1, 5, 12, 15, 18, 20, 22, 23, 25], [0, 2, 3, 4, 5, 6, 8, 11, 13, 14, 18, 21, 23, 24], [0, 28], [6, 12, 20], [7, 8, 9, 13, 15, 21, 24, 28], [15, 17, 29], [2, 3, 4, 10, 13, 21, 22, 26], [7, 11, 12, 17, 19, 21, 22], [0, 5, 6, 7, 9, 12, 13, 14, 22, 23, 26, 28, 29], [0, 1, 3, 29], [0, 12, 17, 26], [0, 2, 4, 7, 17, 20, 29], [0, 2, 4, 7, 8, 10, 12, 15, 25, 28, 29], [2, 8, 10, 19, 23, 25]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 27, 28, 29]
